In [2]:
from sklearn import metrics
with open("PATH",encoding="utf8") as fh:
    train_lines = fh.readlines()

In [3]:
#tuple each tagged word inside a list
array_tutucu=[]
t_tutucu=[]
for asa in train_lines:
    k=asa.split()
    array_tutucu.append(k)
for a in array_tutucu:
    tu=tuple(a)
    t_tutucu.append(tu)

In [4]:

print(t_tutucu[0:50])

[('ATİLLA', 'B-PERSON'), ('Mutman', 'I-PERSON'), (',', 'O'), ('İzmir', 'B-LOCATION'), ('milletvekili', 'O'), (), ('Gecenin', 'O'), ('erken', 'O'), ('saatlerinde', 'O'), ('de', 'O'), (',', 'O'), ('belli', 'O'), ('olacak', 'O'), ('Avrupa', 'B-LOCATION'), ("'", 'O'), ('nın', 'O'), ('en', 'O'), ('büyük', 'O'), ('ülkesinin', 'O'), ('hükümetini', 'O'), ('kimin', 'O'), ('yöneteceği', 'O'), (), ('Bilgisayarlı', 'O'), ('telefon', 'O'), ('klübesi', 'O'), (), ('Kozmopolit', 'O'), ('bir', 'O'), ('ulus', 'O'), (), ('Yıllık', 'O'), ('10', 'O'), ('milyon', 'O'), ('ton', 'O'), ('kapasitesi', 'O'), ('bulunan', 'O'), ('limana', 'O'), ('150', 'O'), ('bin', 'O'), ('DWT', 'O'), ('tonluk', 'O'), ('gemiler', 'O'), ('yanaşabilecek', 'O'), (), ('Uzanan', 'O'), ('bir', 'O'), ('"', 'O'), ('barış', 'O'), ('eli', 'O')]


In [5]:
tur_array=[]
genel_array=[]
for sss in t_tutucu:
    if len(sss)!=0:
        tur_array.append(sss)
    else :
        genel_array.append(tur_array)
        tur_array=[]
    

In [6]:

token=[]
entity=[]
i=0
for genel in genel_array:
    for tl in genel:
        if len(tl)==2:
            token.append(tl[0])
            entity.append(tl[1])
    
tokens=tuple(token)
entity=tuple(entity) 
print(tokens[0:10])
print(entity[0:10])


('ATİLLA', 'Mutman', ',', 'İzmir', 'milletvekili', 'Gecenin', 'erken', 'saatlerinde', 'de', ',')
('B-PERSON', 'I-PERSON', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O')


In [7]:
#finding the number of word tags at the beginning of the sentence and calculating the probability of their being found
b_location=0
b_organization=0
b_person=1
i_location=0
i_organization=0
i_person=0
o=0
for i in range(len(t_tutucu)-1):
    if len(t_tutucu[i]) < 2:
        if t_tutucu[i+1][1] == "B-LOCATION":
            b_location+=1
        elif t_tutucu[i+1][1] == "B-ORGANIZATION":
            b_organization+=1
        elif t_tutucu[i+1][1] == "B-PERSON":
            b_person+=1
        elif t_tutucu[i+1][1] == "I-LOCATION":
            i_location+=1
        elif t_tutucu[i+1][1] == "I-ORGANIZATION":
            i_organization+=1
        elif t_tutucu[i+1][1] == "I-PERSON":
            i_person+=1
        elif t_tutucu[i+1][1] == "O":
            o+=1
sum=b_location+b_organization+b_person+i_location+i_organization+i_person+o    

In [8]:
import pandas as pd
import numpy as np

from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    nltk_tagged = pos_tag(sentence.split())
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(WordNetLemmatizer().lemmatize(word, tag))
    return lemmatized_sentence   

In [9]:
#creating emission matrix
tokens = [lemmatize_sentence(token.lower())[0] for token in tokens]

df = pd.DataFrame(tokens,entity).reset_index().rename(columns={0:'tokens','index':'entity'})
df['values']=1
df = df.pivot_table(index='entity',columns='tokens',aggfunc='sum').fillna(0)
df = df.div(df.sum(axis=1), axis=0)
df.columns = df.columns.droplevel(0)
print(df)

tokens                 "         $         %         &         '         (  \
entity                                                                       
B-LOCATION      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
B-ORGANIZATION  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
B-PERSON        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
I-LOCATION      0.000000  0.000000  0.000000  0.000000  0.003115  0.000000   
I-ORGANIZATION  0.000000  0.000000  0.000000  0.002730  0.004949  0.000000   
I-PERSON        0.000634  0.000000  0.000000  0.000000  0.001267  0.000000   
O               0.023062  0.000003  0.000027  0.000024  0.046473  0.003584   

tokens                )         +         ,         -  ...   şüpheyi  \
entity                                                 ...             
B-LOCATION      0.00000  0.000000  0.000000  0.000000  ...  0.000000   
B-ORGANIZATION  0.00000  0.000000  0.000000  0.000000  ...  0.000000   
B-PERSON 

In [10]:
#Creating transition matrix
df2 = []
for index in range(len(t_tutucu)-1):
    if len(t_tutucu[index+1]) ==2 and len(t_tutucu[index]) ==2 :
        first_token = t_tutucu[index][1]
        next_token = t_tutucu[index+1][1]
        df2.append({'first_token':first_token,'next_token':next_token})
df2 = pd.DataFrame(df2)
df2['values'] = 1
df2 = df2.pivot_table(index='first_token',columns='next_token',aggfunc='sum').fillna(0)
df2 = df2.div(df2.sum(axis=1), axis=0)
df2.columns = df2.columns.droplevel(0)
df2 = pd.concat([pd.DataFrame(columns=['B-LOCATION','I-LOCATION','O','B-ORGANIZATION','B-PERSON','I-ORGANIZATION','I-PERSON'],index=['<s>'],data=[[b_location/sum,i_location/sum,o/sum,b_organization/sum,b_person/sum,i_organization/sum,i_person/sum]]),df2],axis=0)

print(df2)    

                B-LOCATION  I-LOCATION         O  B-ORGANIZATION  B-PERSON  \
<s>               0.073462    0.000000  0.704629        0.094637  0.127272   
B-LOCATION        0.006657    0.105103  0.885788        0.001985  0.000467   
B-ORGANIZATION    0.008235    0.000000  0.601792        0.002786  0.000484   
B-PERSON          0.002650    0.000000  0.555126        0.001363  0.005982   
I-LOCATION        0.007843    0.301176  0.687059        0.000784  0.003137   
I-ORGANIZATION    0.004281    0.000000  0.535616        0.002568  0.000856   
I-PERSON          0.003838    0.000000  0.896028        0.003338  0.001502   
O                 0.019638    0.000000  0.933962        0.017229  0.029171   

                I-ORGANIZATION  I-PERSON  
<s>                   0.000000  0.000000  
B-LOCATION            0.000000  0.000000  
B-ORGANIZATION        0.386702  0.000000  
B-PERSON              0.000000  0.434878  
I-LOCATION            0.000000  0.000000  
I-ORGANIZATION        0.456678  0.00000

In [11]:
with open("PATH",encoding="utf8") as fh:
    test_lines = fh.readlines()

In [12]:
array_tutucu=[]
t_tutucu=[]
m_test=[]
for asa in test_lines:
    k=asa.split()
    array_tutucu.append(k)
for x in array_tutucu:
    if len(x) ==2:
        m_test.append(x[1])
print(len(m_test))    

49600


In [13]:
testMetin=""
for test in array_tutucu:
    if len(test)==2:
        testMetin = testMetin + test[0] +" "



In [14]:
test_1= testMetin
list_of_entity = []
previous = "<s>"
tokens = [lemmatize_sentence(token.lower())[0] for token in word_tokenize(test_1)]

In [15]:
import pandas as pd
  
#Tokens that don't exist in transition matrix assigned random values
dat = [[0.01], [0.01], [0.01],[0.01], [0.01], [0.01],[0.01]]
  
# Create the pandas DataFrame
df3 = pd.DataFrame(dat, columns=['Word'])
  
for token in tokens:
    if token in list(df.columns.values):
        matrix = df[token].multiply(df2.loc[previous])
        previous = df.index[np.argmax(matrix)]
        token_label_pair = (token,previous)
        list_of_entity.append(token_label_pair)
    else:
        matrix = df3["Word"].multiply(df2.loc[previous])
        previous = df.index[np.argmax(matrix)]
        token_label_pair = (token,previous)
        list_of_entity.append(token_label_pair)
        

In [16]:
pred=[]
for x in list_of_entity:
    if len(x) ==2:
        pred.append(x[1])


In [17]:
print(len(m_test))
print(len(pred))


49600
49600


In [18]:
print(metrics.classification_report(m_test, pred))

                precision    recall  f1-score   support

    B-LOCATION       0.79      0.86      0.82      1126
B-ORGANIZATION       0.85      0.55      0.67       873
      B-PERSON       0.89      0.75      0.81      1603
    I-LOCATION       0.75      0.33      0.46       211
I-ORGANIZATION       0.63      0.15      0.24       864
      I-PERSON       0.91      0.52      0.67       803
             O       0.96      1.00      0.98     44120

      accuracy                           0.95     49600
     macro avg       0.83      0.59      0.66     49600
  weighted avg       0.95      0.95      0.94     49600



In [19]:
#Another test data
with open("PATH",encoding="utf8") as fh:
    test_lines = fh.readlines()

In [20]:
array_tutucu=[]
t_tutucu=[]
m_test=[]
for asa in test_lines:
    k=asa.split()
    array_tutucu.append(k)
for x in array_tutucu:
    if len(x) ==2:
        m_test.append(x[1])
print(len(m_test))  

45532


In [21]:
testMetin=""
for test in array_tutucu:
    if len(test)==2:
        testMetin = testMetin + test[0] +" "

In [22]:
test_1= testMetin
list_of_entity = []
previous = "<s>"
tokens = [lemmatize_sentence(token.lower())[0] for token in word_tokenize(test_1)]

In [23]:
import pandas as pd
  
# initialize list of lists
dat = [[0.01], [0.01], [0.01],[0.01], [0.01], [0.01],[0.01]]
  
# Create the pandas DataFrame
df3 = pd.DataFrame(dat, columns=['Word'])
  
for token in tokens:
    if token in list(df.columns.values):
        matrix = df[token].multiply(df2.loc[previous])
        previous = df.index[np.argmax(matrix)]
        token_label_pair = (token,previous)
        list_of_entity.append(token_label_pair)
    else:
        matrix = df3["Word"].multiply(df2.loc[previous])
        previous = df.index[np.argmax(matrix)]
        token_label_pair = (token,previous)
        list_of_entity.append(token_label_pair)
        

In [24]:
pred=[]
for x in list_of_entity:
    if len(x) ==2:
        pred.append(x[1])

In [25]:
print(len(m_test))
print(len(pred))

45532
45532


In [26]:
print(metrics.classification_report(m_test, pred))

                precision    recall  f1-score   support

    B-LOCATION       0.81      0.92      0.86       942
B-ORGANIZATION       0.87      0.66      0.75       842
      B-PERSON       0.90      0.86      0.88      1400
    I-LOCATION       0.68      0.53      0.60       107
I-ORGANIZATION       0.68      0.25      0.37       589
      I-PERSON       0.90      0.68      0.77       680
             O       0.98      0.99      0.99     40972

      accuracy                           0.97     45532
     macro avg       0.83      0.70      0.75     45532
  weighted avg       0.96      0.97      0.96     45532

